In [1]:
import pandas as pd
import tinvest as ti
from tinvest import CandleResolution
from tinvest import TooManyRequestsError, UnexpectedError

from threading import Thread
from tqdm.notebook import tqdm

import os
import datetime as dt
import time

import sqlite3
import investpy
import yfinance as yf

In [2]:
pd.options.display.max_columns = 150

### Источники данных
Рассматривалось использование трех основных источников информации об акциях в качестве возможных: 
- tinvest, SDK для Tinkoff Инвестиции API
- investpy, библиотека для использования данных с сайта investing.com
- yfinanse, библиотека для и использования yahoo finanse API

В связи с большим количеством необходимых запросов была задействевоно использование локальной БД с помощью sqlite3

#### API Tinkoff Инвестиции
Акуальная документация соответсвует версии библиотек 3.0 и выше, для которых необходимо использование python версией выше 3.8, поэтому для запуска кода, использующего эту библиотеку необходимо использование указанной версии python.

Для использования API необходимо наличие токена, который можно завести в личном кабинете Тинькофф. Ограничения API - 240 операций в секунду на запросы по данным рынка. Для работы с API использовалось неофициальное SDK - tinvest - упомянутое на официальной странице API.

Примера данных представлен также в файлах tinvest_daily_data.csv и tinvest_stocks_info.xlsx в приложенной ссылке на Google Drive.

In [22]:
TOKEN = os.getenv('TINVEST_TOKEN')

In [23]:
# client = ti.SyncClient(TOKEN)
# response = client.get_market_stocks()
# result = response.dict()["payload"]["instruments"]
# candle = client.get_market_candles(
#     "BBG000B9X8C0",
#     "2020-06-01T00:00:00+03:00",
#     "2020-06-30T00:00:00+03:00",
#     CandleResolution("day"),
# ).dict()["payload"]["candles"]

# ti_fundamental_df = pd.DataFrame(result)
# candle_df = pd.DataFrame(candle)
# display(ti_fundamental_df.head())
# display(candle_df.head())

,currency,figi,isin,lot,min_price_increment,name,ticker,type,min_quantity
0,Currency.usd,BBG000HLJ7M4,US45867G1013,1,0.01,InterDigItal Inc,IDCC,InstrumentType.stock,NaN
1,Currency.usd,BBG000BDSLD7,US8030542042,1,0.01,АДР SAP,SAP,InstrumentType.stock,NaN
2,Currency.usd,BBG002293PJ4,US74967X1037,1,0.01,RH,RH,InstrumentType.stock,NaN
3,Currency.usd,BBG000CTQBF3,US8552441094,1,0.01,Starbucks Corporation,SBUX,InstrumentType.stock,NaN
4,Currency.usd,BBG000BH0FR6,US81181C1045,1,0.01,Seagen Inc.,SGEN,InstrumentType.stock,NaN


,c,figi,h,interval,l,o,time,v
0,74.37,BBG000B9X8C0,75.68,CandleResolution.day,73.86,75.14,2020-06-01 07:00:00+00:00,293707
1,74.91,BBG000B9X8C0,75.25,CandleResolution.day,74,74.38,2020-06-02 07:00:00+00:00,283474
2,76.08,BBG000B9X8C0,76.21,CandleResolution.day,75.08,75.21,2020-06-03 07:00:00+00:00,408469
3,73.99,BBG000B9X8C0,76.5,CandleResolution.day,73.1,75.66,2020-06-04 07:00:00+00:00,282975
4,75.58,BBG000B9X8C0,76.96,CandleResolution.day,74.44,74.96,2020-06-05 07:00:00+00:00,411271


Что значат эти исторические данные: 
- close price (c) цена бумаги на момент окончания рассмотренного периода
- open price (o) цена бумаги на момент начала рассмотренного периода
- high price (h) наибольшая цена за рассмотренный период
- low price (l) наименьшая цена за рассмотренный период

Заполнение БД из этого источника осуществляется с помощью скриптов в папке db.

In [5]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

In [6]:
database = r"pythonsqlite.db"
conn = create_connection(database)

2.6.0


In [7]:
stocks = pd.read_sql(sql='Select * from stocks', con=conn)

In [8]:
stocks.head()

,UPDATE_DATE,CURRENCY,FIGI,ISIN,LOT,MIN_PRICE_INCREMENT,NAME,TICKER,TYPE,MIN_QUANTITY
0,2021-06-10 15:18:40.216564,USD,BBG000HLJ7M4,US45867G1013,1,0.01,InterDigItal Inc,IDCC,Stock,NaN
1,2021-06-10 15:18:40.223985,USD,BBG000BDSLD7,US8030542042,1,0.01,АДР SAP,SAP,Stock,NaN
2,2021-06-10 15:18:40.231109,USD,BBG002293PJ4,US74967X1037,1,0.01,RH,RH,Stock,NaN
3,2021-06-10 15:18:40.238816,USD,BBG000CTQBF3,US8552441094,1,0.01,Starbucks Corporation,SBUX,Stock,NaN
4,2021-06-10 15:18:40.245532,USD,BBG000BPL8G3,US5537771033,1,0.01,MTS Systems Corp,MTSC,Stock,NaN


#### Investing.com
У сайта есть свой интерфейс по выгрузке данных, с которым можно ознакомится, используя библиотеку investpy. Основной плюс - стабильность работы. API поддерживается бесплатно и официально.

In [9]:
i_com_fundamentals_df = investpy.get_stocks()

sample = stocks.sample(5)

Рынок бумаг Тинькоф значительно меньше чем глобальный рынок, что приносит свои проблемы в интерпретации данных, которые можно получить от investing.com. По одному тикеру можно получить сразу несколько позиций, связанных с разными странами, а то и разными компаниями, в то время как в данных от Тинькофф тикеры уникальны. 

Скорей всего тут есть своя, пока не известная мне специфика, поэтому на текущий момент будут рассматриваться только данные, связанные с бумагами доступными через инструменты Тинькофф.

In [10]:
print('Ценных бумаг в доступе Тинькофф Инвестиции - {0} штук'.format(len(stocks)))
print('Ценных бумаг в базе Investing.com - {0} штук'.format(len(i_com_fundamentals_df)))

Ценных бумаг в доступе Тинькофф Инвестиции - 1695 штук
Ценных бумаг в базе Investing.com - 39952 штук


In [11]:
i_com_fundamentals_df.merge(sample, how = 'inner', left_on = ['currency', 'symbol'], right_on = ['CURRENCY', 'TICKER'])

,country,name,full_name,isin,currency,symbol,UPDATE_DATE,CURRENCY,FIGI,ISIN,LOT,MIN_PRICE_INCREMENT,NAME,TICKER,TYPE,MIN_QUANTITY
0,united states,Abbott Labs,Abbott Laboratories,US0028241000,USD,ABT,2021-06-10 15:18:53.905824,USD,BBG000B9ZXB4,US0028241000,1,0.01,Abbott,ABT,Stock,NaN
1,united states,John Bean Tech,John Bean Technologies Corporation,US4778391049,USD,JBT,2021-06-10 15:18:53.753336,USD,BBG000VLBCQ1,US4778391049,1,0.01,John Bean Technologies,JBT,Stock,NaN
2,united states,Prestige Brand,Prestige Brand Holdings Inc,US74112D1019,USD,PBH,2021-06-10 15:18:44.011135,USD,BBG000QM0RZ4,US74112D1019,1,0.01,Prestige Consumer Healthcare I,PBH,Stock,NaN
3,united states,Lear,Lear Corporation,US5218652049,USD,LEA,2021-06-10 15:18:45.836699,USD,BBG000PTLGZ1,US5218652049,1,0.01,Lear Corp,LEA,Stock,NaN
4,united states,CoreLogic,Corelogic Inc,US21871D1037,USD,CLGX,2021-06-10 15:18:55.522599,USD,BBG000C2H3F0,US21871D1037,1,0.01,CoreLogic Inc United States,CLGX,Stock,NaN


In [ ]:
i_com_fundamentals_df.merge(sample, how = 'inner', left_on = ['currency', 'symbol'], right_on = ['CURRENCY', 'TICKER'])

In [12]:
i_com_fundamentals_df[i_com_fundamentals_df['symbol'].isin(sample['TICKER'])]

,country,name,full_name,isin,currency,symbol
1091,canada,Absolute Software,Absolute Software Corporation,CA00386B1094,CAD,ABT
1312,canada,Premium Brands,Premium Brands Holdings Corporation,CA74061A1084,CAD,PBH
2836,mexico,Abbott Labs,Abbott Laboratories,US0028241000,MXN,ABT
3074,mexico,Lear,Lear Corp,US5218652049,MXN,LEA
3609,united states,Abbott Labs,Abbott Laboratories,US0028241000,USD,ABT
5211,united states,John Bean Tech,John Bean Technologies Corporation,US4778391049,USD,JBT
5219,united states,Prestige Brand,Prestige Brand Holdings Inc,US74112D1019,USD,PBH
7589,united states,Lear,Lear Corporation,US5218652049,USD,LEA
7600,united states,CoreLogic,Corelogic Inc,US21871D1037,USD,CLGX
10354,germany,CoreLogic,Corelogic Inc,US21871D1037,EUR,CLGX


Через данный сервис также можно получать информацию об исторических продажах, что однако ограничено дневными, недельными и месячными срезами, в отличии от двух других источников.

In [14]:
appl_daily_df = investpy.get_stock_historical_data(stock='AAPL',
                                        country='United States',
                                        from_date='01/01/2010',
                                        to_date='01/01/2020', interval='Daily')

In [15]:
appl_daily_df.head()

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2010-01-04,7.62,7.66,7.58,7.64,493728704,USD
2010-01-05,7.66,7.70,7.62,7.66,601904640,USD
2010-01-06,7.66,7.69,7.53,7.53,552158912,USD
2010-01-07,7.56,7.57,7.47,7.52,477129760,USD
2010-01-08,7.51,7.57,7.47,7.57,447876768,USD


#### Yahoo Finance API
Позволяет получить информацию используя данные сайта Yahoo Finance. Существуют как платные, так и бесплатные инструментарии для работы с эти API. Для ограниченной выгрузки данных можно использовать бесплатный, одним из которых является библиотека yfinance

In [16]:
aapl = yf.Ticker('aapl')

In [17]:
aapl.info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 100000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

In [18]:
aapl.history(
    interval='1h', 
    start="2021-06-02", 
    end= "2021-06-03"
)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2021-06-02 09:30:00-04:00,124.519997,125.190002,124.220001,125.084999,16148036,0,0
2021-06-02 10:30:00-04:00,125.079697,125.239998,124.650002,124.970001,8434366,0,0
2021-06-02 11:30:00-04:00,124.970001,125.180000,124.620003,124.706802,5315912,0,0
2021-06-02 12:30:00-04:00,124.705002,124.709999,124.070000,124.195000,5513431,0,0
2021-06-02 13:30:00-04:00,124.199997,124.510002,124.050003,124.480003,5447741,0,0
2021-06-02 14:30:00-04:00,124.485001,124.794998,124.410004,124.785004,5563554,0,0
2021-06-02 15:30:00-04:00,124.790001,125.168999,124.660004,125.089996,6289304,0,0


Основная проблема данного источника - в его содержимом под капотом. Данная библиотека использует скрапинг данных с сайта, изменение в структуре которого могут привести к неадекватной работе сервиса.

### Подитог
- Исторические данные-свечки использовальзуются от сервиса Tinkoff Инвестиции, так как несмотря на ограничения запросов, его данные наиболее соответсвуют продажам через российских брокеров и могут рассматривать часовые данные. 
- Справочные данные будут использоваться от сервиса yahoo finance, наиболее "богатый" на информацию из рассмотренных источников
- Период исторических данных - 1 год, с 01.06.2020 по 31.05.2021